In [1]:
# Trying out a new dataset

In [3]:
import numpy as np
import pandas as pd

In [4]:
import os
os.listdir('input')

['ability_ids.csv',
 'ability_upgrades.csv',
 'chat.csv',
 'cluster_regions.csv',
 'hero_names.csv',
 'item_ids.csv',
 'match.csv',
 'match_outcomes.csv',
 'objectives.csv',
 'patch_dates.csv',
 'players.csv',
 'player_ratings.csv',
 'player_time.csv',
 'purchase_log.csv',
 'teamfights.csv',
 'teamfights_players.csv',
 'test_labels.csv',
 'test_player.csv',
 'yasp_sample.json']

matches: contains top level information about each match. see https://wiki.teamfortress.com/wiki/WebAPI/GetMatchDetails#Tower_Status%22tower_status_dire%22:%202047) for interpreting tower and barracks status. Cluster can link matches to geographic region.

In [6]:
matches = pd.read_csv('input/match.csv')
matches.tail()

,match_id,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster
49995,49995,1447829192,3025,1824,0,0,63,117,22,True,0,0,111
49996,49996,1447829181,1451,1982,384,48,63,147,22,True,0,0,204
49997,49997,1447829197,2672,1830,0,0,63,130,22,True,0,0,111
49998,49998,1447829189,2211,1982,4,3,63,645,22,True,0,0,151
49999,49999,1447829215,2788,0,2046,63,0,326,22,False,0,0,154


match_id has been reencoded from steams  match_id to save a little but of space. tower_status and barracks_status are binarymasks indicating whether various structures have been destroyed. See the very bottom of https://wiki.teamfortress.com/wiki/WebAPI/GetMatchDetails#Tower_Status%22tower_status_dire%22:%202047, for details. radiant_win would be good choice for a target of a binary classification task.

In [7]:
# Each file needs to be removed after use.
del matches
gc.collect()

373

players: Individual players are identified by account_id but there is an option to play anonymously and roughly one third of the account_id are not available. Anonymous users have the value of 0 for account_id. Contains totals for kills, deaths, denies, etc. Player action counts are available, and are indicated by variable names beginning with unit_order_. Counts for reasons for acquiring or losing gold, and gaining experience, have prefixes gold_, and xp_.

In [8]:
players = pd.read_csv('input/players.csv')
players.iloc[:5,:15]

,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,assists,denies,last_hits,stuns,hero_damage
0,0,0,86,0,3261,10960,347,362,9,3,18,1,30,76.7356,8690
1,0,1,51,1,2954,17760,494,659,13,3,18,9,109,87.4164,23747
2,0,0,83,2,110,12195,350,385,0,4,15,1,58,None,4217
3,0,2,11,3,1179,22505,599,605,8,4,19,6,271,None,14832
4,0,3,67,4,3307,23825,613,762,20,3,17,13,245,None,33740


account_id is useful if you want to look at multiple matches for the same player but be aware that many players choose to hide their account_id so it won't be available.

In [9]:
players['account_id'].value_counts()

0         181169
2701          71
2362          60
10307         59
18680         59
4390          58
2962          57
6551          54
12823         54
2487          54
14457         54
22329         53
9022          51
2312          51
836           51
20992         51
15512         50
35210         50
6647          49
14616         49
3284          49
19481         48
2032          47
12066         47
12125         47
2437          47
9527          47
22466         46
28049         46
53928         46
           ...  
35429          1
37478          1
39527          1
60009          1
121423         1
119374         1
117325         1
127562         1
141873         1
143922         1
145971         1
131636         1
133685         1
53975          1
137783         1
156216         1
158265         1
148028         1
150077         1
62163          1
107072         1
109121         1
64210          1
113219         1
98884          1
100933         1
102982         1
105031        

About 180000 thousand of the records out of a possible 500k are not available. The data was sampled using a time based split so this distribution should be representative of the rest of the available data.

In [10]:
players.iloc[:5,20:30]

,item_3,item_4,item_5,level,leaver_status,xp_hero,xp_creep,xp_roshan,xp_other,gold_other
0,56,108,0,16,0,8840.0,5440.0,NaN,83.0,50.0
1,102,24,108,22,0,14331.0,8440.0,2683.0,671.0,395.0
2,108,65,0,17,0,6692.0,8112.0,NaN,453.0,259.0
3,164,79,160,21,0,8583.0,14230.0,894.0,293.0,100.0
4,0,137,63,24,0,15814.0,14325.0,NaN,62.0,NaN


In [11]:
players.iloc[:5,40:55]

,unit_order_move_to_position,unit_order_move_to_target,unit_order_attack_move,unit_order_attack_target,unit_order_cast_position,unit_order_cast_target,unit_order_cast_target_tree,unit_order_cast_no_target,unit_order_cast_toggle,unit_order_hold_position,unit_order_train_ability,unit_order_drop_item,unit_order_give_item,unit_order_pickup_item,unit_order_pickup_rune
0,4070.0,1.0,25.0,416.0,51.0,144.0,3.0,71.0,NaN,188.0,16.0,NaN,NaN,NaN,2.0
1,5894.0,214.0,165.0,1031.0,98.0,39.0,4.0,439.0,NaN,346.0,22.0,NaN,NaN,12.0,52.0
2,7053.0,3.0,132.0,645.0,36.0,160.0,20.0,373.0,NaN,643.0,17.0,5.0,NaN,7.0,8.0
3,4712.0,133.0,163.0,690.0,9.0,15.0,7.0,406.0,NaN,150.0,21.0,NaN,NaN,1.0,9.0
4,3853.0,7.0,7.0,1173.0,31.0,84.0,8.0,198.0,NaN,111.0,23.0,1.0,NaN,NaN,2.0


I am pretty sure these are counts of user issued commands. The only reason they look like floats is because of how pandas handles nan. It is a float so if there is even one in a column all other numbers are converted. There may be a way of approximating actions per minute(not quite clicks per minute) from this.

In [12]:
#cleanup
del players
gc.collect()

166

player_time: Contains last hits, experience, and gold sampled at one minute interval for all players in all matches. The column names indicate the player_slot. For instance xp_t_1 indicates that this column has experience sums for the player in slot one.

In [13]:
player_time = pd.read_csv('input/player_time.csv')
player_time.head()

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,60,409,0,63,142,1,186,168,0,...,62,345,6,351,100,0,77,613,1,125
2,0,120,546,0,283,622,4,645,330,0,...,543,684,12,805,200,0,210,815,5,323
3,0,180,683,1,314,927,9,1202,430,0,...,842,958,16,1135,300,0,210,1290,8,527
4,0,240,956,1,485,1264,11,1583,530,0,...,1048,1500,26,1842,400,0,210,1431,9,589


In [16]:
a_match = player_time.query('match_id == 2')
a_match.T

,89,90,91,92,93,94,95,96,97,98,...,127,128,129,130,131,132,133,134,135,136
match_id,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
times,0,60,120,180,240,300,360,420,480,540,...,2280,2340,2400,2460,2520,2580,2640,2700,2760,2820
gold_t_0,0,174,400,665,902,1140,1609,2039,2271,2461,...,9394,9494,9846,10008,10192,10427,10634,10734,10762,10762
lh_t_0,0,2,5,9,10,12,13,14,17,17,...,75,75,83,86,88,91,97,97,97,97
xp_t_0,0,93,288,556,1084,1291,1597,1927,2135,2357,...,10259,10259,10642,10717,10810,11203,11285,11310,11310,11310
gold_t_1,0,280,589,858,1000,1518,1736,1873,2010,2234,...,12959,13059,13241,13881,13981,14193,14475,14575,14603,14603
lh_t_1,0,2,7,11,12,14,17,18,19,22,...,157,157,159,165,165,170,179,179,179,179
xp_t_1,0,337,791,1214,1410,1795,2187,2373,2538,2889,...,15096,15096,15216,15883,15883,16073,16235,16235,16235,16235
gold_t_2,0,99,241,432,963,1103,1246,1566,1937,2561,...,14116,14453,14817,15251,15351,15801,15952,16070,16098,16098
lh_t_2,0,0,1,2,5,6,7,9,15,19,...,115,123,132,138,138,150,153,154,154,154


Since each match lasts for a different amount of time storing them with time on the horizontal axis would take a lot of space. The suffix for each variable indicates the value of the player_slot variable allowing this data to be combined with players.csv if desired.

In [17]:
del player_time
gc.collect()

736

teamfights: Start and stop time of teamfights, as well as last death time. Teamfights appear to be all battles with three or more deaths. As such this does not include all battles for the entire match.

In [18]:
teamfights = pd.read_csv('input/teamfights.csv')
teamfights.head()

,match_id,start,end,last_death,deaths
0,0,220,252,237,3
1,0,429,475,460,3
2,0,900,936,921,3
3,0,1284,1328,1313,3
4,0,1614,1666,1651,5


start, end and last_death contain the time for those events. Each row contains very basic info about each team fight. Time is in seconds. I was considering adding a specific column for the count of teamfights in a match. It would make getting the first teamfight for each match easier.

In [19]:
del teamfights
gc.collect()

266

teamfights_players : Additional information provided for each player in each teamfight. player_slot can be used to link this back to players.csv

In [20]:
teamfights_players = pd.read_csv('input/teamfights_players.csv')
teamfights_players.head()

,match_id,player_slot,buybacks,damage,deaths,gold_delta,xp_end,xp_start
0,0,0,0,105,0,173,536,314
1,0,1,0,566,1,0,1583,1418
2,0,2,0,0,0,0,391,391
3,0,3,0,0,0,123,1775,1419
4,0,4,0,444,0,336,1267,983


Each row in the teamfights.csv corrosponds to ten rows in this file. I have marked this file and teamfights to be updated with specific variable indicating which teamfight in the match it belongs to this should make joining and working with these tables easier.

In [21]:
del teamfights_players
gc.collect()

404

objectives: Gives information on all the objectives completed, by which player and at what time.

In [22]:
objectives = pd.read_csv('input/objectives.csv')
objectives.head()

,match_id,key,player1,player2,slot,subtype,team,time,value
0,0,NaN,0,6,0.0,CHAT_MESSAGE_FIRSTBLOOD,NaN,1,309
1,0,NaN,3,-1,3.0,CHAT_MESSAGE_TOWER_KILL,2.0,894,2
2,0,NaN,2,-1,NaN,CHAT_MESSAGE_ROSHAN_KILL,2.0,925,200
3,0,NaN,1,-1,1.0,CHAT_MESSAGE_AEGIS,NaN,925,0
4,0,NaN,7,-1,7.0,CHAT_MESSAGE_TOWER_KILL,3.0,1016,3


In [23]:
del objectives
gc.collect()

23

chat: All chat for the 50k matches. There is plenty of profanity, and good natured trolling.

In [25]:
chat = pd.read_csv('input/chat.csv')
chat.head()

,match_id,key,slot,time,unit
0,0,force it,6,-8,6k Slayer
1,0,space created,1,5,Monkey
2,0,hah,1,6,Monkey
3,0,ez 500,6,9,6k Slayer
4,0,mvp ulti,4,934,Kira


test_labels: match_id and radiant_win(as integer 1 or 0)

In [27]:
del chat
gc.collect()

240

In [26]:
test_labels = pd.read_csv('input/test_labels.csv')
test_labels.head()

,match_id,radiant_win
0,50000,0
1,50001,1
2,50002,0
3,50003,1
4,50004,0


In [28]:
del test_labels
gc.collect()

11

test_player: full player and match table with hero_id, player_slot, match_id, and account_id

In [29]:
# problem with the hero_ids in test_player brought to my attention by @Dexter, thanks!
# hero_id is 0 in 15 cases. 

test_players = pd.read_csv('input/test_player.csv')
hero_names = pd.read_csv('input/hero_names.csv')

## My comments: Maybe the 0 hero_id's correspond to the players who failed to pick a hero in the beginning of the game

In [30]:
# As can been seen the number of zeros appearing here are much less then the least popular hero. These are very likely
# caused by processing problems, either in my data generation code, or in the data pulled from steam. 
test_players['hero_id'].value_counts().tail()

65     1763
89     1715
103    1475
66     1091
0        15
Name: hero_id, dtype: int64

In [31]:
test_players.query('hero_id == 0')

,match_id,account_id,hero_id,player_slot
92509,59250,66273,0,132
112699,61269,0,0,132
119238,61923,0,0,131
224493,72449,205379,0,3
240264,74026,0,0,4
270509,77050,0,0,132
274501,77450,5008,0,1
371278,87127,21848,0,131
376299,87629,0,0,132
395916,89591,238426,0,129


No pattern immediately jumps out in relationship to the missing hero IDs. Except maybe they are more common for Dire. The safest way to deal with this is probably to remove the the matches in which any hero_id is 0

In [33]:
# remove matches with any invalid hero_ids
# imputing hero_id, is likely possible but the data is not available online in this dataset

matches_with_zero_ids = test_players.query('hero_id == 0')['match_id'].values.tolist()
test_players = test_players.query('match_id != @matches_with_zero_ids')

In [34]:
# check that the invalid ids are removed
# This is now on my list of bugs to fix for next release. 
test_players['hero_id'].value_counts().tail()

78     1768
65     1762
89     1715
103    1475
66     1091
Name: hero_id, dtype: int64

In [35]:
del test_players
del hero_names
gc.collect()

98

player_ratings contains match counts, win counts, and TrueSkill rating, calculated on 900k matches which occurred prior to other uploaded data. trueskill ratings have two components, mu, which can be interpreted as the skill, with higher value being better, and sigma which is the uncertainty of the rating.

A possible way to measure skill rating when we don't have MMR data See this kernel for details on how this was calculated https://www.kaggle.com/devinanzelmo/d/devinanzelmo/dota-2-matches/dota-2-skill-rating-with-trueskill

In [38]:
# player_ratings.csv contains trueskill ratings for players in the match, and test data.
# True Skill is a rating method somewhat like MMR, and can be used to sort players by skill. 

player_ratings = pd.read_csv('input/player_ratings.csv')
player_ratings.head()

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,236579,14,24,27.868035,5.212361
1,-343,1,1,26.544163,8.065475
2,-1217,1,1,26.521103,8.114989
3,-1227,1,1,27.248025,8.092217
4,-1284,0,1,22.931016,8.092224


In [40]:
# Now create a list of player rankings by using the formula mu - 3*sigma
# This ranking formula penalizes players with fewer matches because there is more uncertainty

player_ratings['conservative_skill_estimate'] = player_ratings['trueskill_mu'] - 3*player_ratings['trueskill_sigma']
player_ratings.head()

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma,conservative_skill_estimate
0,236579,14,24,27.868035,5.212361,12.230953
1,-343,1,1,26.544163,8.065475,2.347736
2,-1217,1,1,26.521103,8.114989,2.176136
3,-1227,1,1,27.248025,8.092217,2.971375
4,-1284,0,1,22.931016,8.092224,-1.345657


In [41]:
player_ratings = player_ratings.sort_values(by='conservative_skill_estimate', ascending=False)
# negative account ids are players not appearing in other data available in this dataset.

player_ratings.head(10)

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma,conservative_skill_estimate
408270,19042,121,180,41.005953,1.943824,35.174481
158745,-74329252,31,33,48.825892,4.806721,34.405728
499575,-136222868,54,56,46.183549,4.864634,31.589648
352452,57008,25,28,45.434024,4.667524,31.431453
143565,1543,49,59,41.757517,3.554011,31.095486
204236,87263,22,22,46.719826,5.305605,30.803010
305895,1409,102,145,37.703512,2.388866,30.536913
384071,-117081714,27,32,43.188114,4.234263,30.485327
699922,211968,29,29,46.194220,5.348935,30.147414
777531,37301,43,51,41.536257,3.827072,30.055041


In [42]:
del player_ratings
gc.collect()

478

match_outcomes data for ~900k matches used to calculate player ratings. Use this to improve on the ratings I uploaded.

Useful for creating custom skill calculations. Contains results with account_ids for 900k matches occuring prior to the other available data. Data is mostly from patch 6.85 and some from 6.84

In [45]:
match_outcomes = pd.read_csv('input/match_outcomes.csv')
match_outcomes.head()
# each match has data on two rows. the 'rad' tells whether the team is Radiant or not(1 is Radiant 0 is Dire)
# negative account ids are not in the other available data. account_id 0 is for anonymous players.

,match_id,account_id_0,account_id_1,account_id_2,account_id_3,account_id_4,start_time,parser_version,win,rad
0,1636204962,34549,0,0,-51743434,-120875154,1437014585,12,1,0
1,1636204962,0,61598,138825,0,207232,1437014585,12,0,1
2,1636322679,0,-44943233,-240360907,19599,0,1437019968,12,0,0
3,1636322679,-97530201,0,0,0,-116349387,1437019968,12,1,1
4,1637385965,0,0,0,104738,0,1437052551,12,1,0


In [46]:
del match_outcomes
gc.collect()

474

ability_upgrade ability upgrade times and levels

ability_ids use with ability_upgrades.csv to get the names of upgraded abilities

ability_upgrades.csv contains the upgrade performed at each level for each player. ability_ids.csv links the ability ids to the english names of abilities.

In [47]:
ability_upgrades = pd.read_csv('input/ability_upgrades.csv')
ability_ids = pd.read_csv('input/ability_ids.csv')

In [48]:
ability_ids.head()

,ability_id,ability_name
0,0,ability_base
1,5001,default_attack
2,5002,attribute_bonus
3,5003,antimage_mana_break
4,5004,antimage_blink


In [49]:
ability_upgrades.head()

,ability,level,time,player_slot,match_id
0,5448,1,326,0,0
1,5450,2,452,0,0
2,5450,3,582,0,0
3,5448,4,804,0,0
4,5450,5,916,0,0


In [50]:
del ability_upgrades, ability_ids
gc.collect()

397

purchase_log item purchase times

item_ids use with purchase_log.csv to get the names of purchased items

purchase_log.csv contains the time for each purchase. item_ids.csv contains numeric id's for items and the english names

In [51]:
purchase_log = pd.read_csv('input/purchase_log.csv')
item_ids = pd.read_csv('input/item_ids.csv')

In [52]:
item_ids.head()

,item_id,item_name
0,1,blink
1,2,blades_of_attack
2,3,broadsword
3,4,chainmail
4,5,claymore


In [53]:
purchase_log.head()

,item_id,time,player_slot,match_id
0,44,-81,0,0
1,29,-63,0,0
2,43,6,0,0
3,84,182,0,0
4,46,197,0,0


In [54]:
del purchase_log, item_ids
gc.collect()

351

cluster_region allows the mapping cluster found in match.csv to geographic region.

In [55]:
cluster_region = pd.read_csv('input/cluster_regions.csv')
cluster_region.head()

,cluster,region
0,111,US WEST
1,112,US WEST
2,113,US WEST
3,121,US EAST
4,122,US EAST


patch_dates release dates for various patches, use start_time from match.csv to determine which patch a match was played in.

In [56]:
patch_dates = pd.read_csv('input/patch_dates.csv')
patch_dates.head()

,patch_date,name
0,2010-12-24T00:00:00Z,6.70
1,2011-01-21T00:00:00Z,6.71
2,2011-04-27T00:00:00Z,6.72
3,2011-12-24T00:00:00Z,6.73
4,2012-03-10T00:00:00Z,6.74


In [57]:
del cluster_region, patch_dates
gc.collect()

503